In [ ]:
# Asegura que la carpeta raíz del proyecto esté en el sistema de paths de Python
import sys
import os

# Ruta absoluta a la raíz del proyecto
proyecto_root = os.path.abspath(os.path.join(".."))  # .. porque estás en notebooks/
if proyecto_root not in sys.path:
    sys.path.append(proyecto_root)

from src.preprocessing import load_data, clean_data, feature_engineering, scale_and_split

# Ejemplo de uso rápido:
data_path = "../data/raw/sensor_data.csv"

# Cargamos los datos desde un archivo CSV
df = load_data(data_path)

# Eliminamos datos faltantes si los hubiera
df = clean_data(df)

# Creamos nuevas variables basadas en el timestamp
df = feature_engineering(df)

# Escalamos los datos y los dividimos en entrenamiento y prueba
X_train_scaled, X_test_scaled, y_train, y_test = scale_and_split(df, target='falla')
